In [16]:
# -*- coding: utf-8 -*-
import unicodecsv
import os
import pyLDAvis
import pyLDAvis.gensim



from nltk import RegexpTokenizer
from lib.utils import *
from itertools import chain

In [5]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "soya_month.csv"
args.stemmer = "lemma"
args.dictionary = "english"
args.num_iter = 50
args.num_top_words = 8
args.num_topics = 10
args.model = "lda"
args.override = False


dir = os.getcwd()
model_dir = os.path.join(dir, 'models/')
dataset_dir = os.path.join(dir, 'datasets/')
dictionary_dir = os.path.join(dir, 'dictionaries/')
executable_dir = os.path.join(dir, 'executables/')
my_timeslices = [500,500,500,500,500, 346]

model_filename = os.path.join(model_dir, get_model_with_arguments_filename(args))
is_english_word = load_from_dictionary(args.dictionary)

print model_filename

/Users/Hii/Projects/news_scraper/models/soya_month_lemma_50_8_10_lda_english


In [8]:
dataset_filepath = os.path.join(dataset_dir, args.filename)
f = open(dataset_filepath)
reader = unicodecsv.reader(f, encoding='utf-8')

identifiers = reader.next()
contents_idx = identifiers.index("contents")
title_idx = identifiers.index("title")

contents, titles = zip(*[(row[contents_idx], row[title_idx]) for row in reader])

texts = preprocess(contents, args.stemmer, is_english_word)

dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

my_corpus = [dictionary.doc2bow(text) for text in texts]

In [9]:
try:
  model = load_model(model_filename, args.model)
  if args.override:
    raise IOError("Override flag set")
except IOError:
  print "Generating model ..."
  model = generate_model(args.model, my_corpus, dictionary, args.num_topics, args.num_iter)
  model.save(model_filename)

In [10]:
show_topics(args.model, model, args.num_topics, args.num_top_words, titles, my_corpus)

Topic #0 : trade flight encryption phone work destination device pick
Topic #1 : device mini display speaker speed back card thing
Topic #2 : apple device pro ad window i phone gold
Topic #3 : device camera edge galaxy front storage ram battery
Topic #4 : device text watch android world smart feature apple
Topic #5 : service delivery network time box honor order fee
Topic #6 : rice apple device cooker heating mi multitasking year
Topic #7 : mi ram storage snapdragon device processor note power
Topic #8 : plan month data maxi call speed postpaid device
Topic #9 : mobile u good network speed travel back coverage

0 :  This could be the best postpaid plan in town — RM80/month for unlimited everything (top topic: 8)
1 :  iPhone SE and iPad Pro 9.7 official Malaysian pricing revealed (top topic: 2)
2 :  Your WhatsApp conversations are now encrypted end to end (top topic: 0)
3 :  TM UniFi Advance subscription now starts from RM199/month (top topic: 8)
4 :  11street is offering the iPhone SE 

In [18]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, my_corpus, dictionary)



/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      26.951323        1       1  0.145808 -0.073114
8      24.324441        1       2 -0.161891 -0.194634
7       9.880002        1       3  0.164729 -0.094342
4       7.600310        1       4  0.049178  0.033757
2       7.041720        1       5  0.000821  0.002005
1       6.765117        1       6  0.038097  0.068120
9       5.651834        1       7 -0.113504 -0.009482
6       4.679872        1       8  0.028593  0.091241
5       3.576243        1       9 -0.090803  0.083361
0       3.529139        1      10 -0.061028  0.093086, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
1900  Default  177.000000           plan  177.000000  30.0000  30.0000
1548  Default  157.000000          month  157.000000  29.0000  29.0000
338   Default   69.000000            ram   69.000000  28.0000  28.0000
134   Default   43.000000             mi   43.000000  27.0000  27.0000
1906  Default   63.000000         mobile   63.000000  26.0000  26.0000
2406  Default   47.000000          apple   47.000000  25.0000  25.0000
2895  Default   67.000000        storage   67.000000  24.0000  24.0000
357   Default   62.000000           maxi   62.000000  23.0000  23.0000
2797  Default   48.000000       postpaid   48.000000  22.0000  22.0000
2612  Default   63.000000           data   63.000000  21.0000  21.0000
942   Default   34.000000     snapdragon   34.000000  20.0000  20.0000
1352  Default   63.000000          speed   63.000000  19.0000  19.0000
301   Default   46.000000        service   46.000000  18.0000  18.0000
391   Default   17.000000           rice   17.000000  17.0000  17.0000
247   Default   35.000000        network   35.000000  16.0000  16.0000
1012  Default  103.000000         camera  103.000000  15.0000  15.0000
1537  Default   29.000000              u   29.000000  14.0000  14.0000
1439  Default   56.000000      processor   56.000000  13.0000  13.0000
2674  Default  250.000000         device  250.000000  12.0000  12.0000
107   Default   62.000000           call   62.000000  11.0000  11.0000
2184  Default   38.000000      broadband   38.000000  10.0000  10.0000
690   Default   46.000000       offering   46.000000   9.0000   9.0000
2146  Default   34.000000      unlimited   34.000000   8.0000   8.0000
1519  Default   74.000000           time   74.000000   7.0000   7.0000
1131  Default   39.000000            pro   39.000000   6.0000   6.0000
120   Default   43.000000          phone   43.000000   5.0000   5.0000
20    Default   48.000000           spec   48.000000   4.0000   4.0000
193   Default   30.000000          order   30.000000   3.0000   3.0000
2416  Default   20.000000            fee   20.000000   2.0000   2.0000
2535  Default   70.000000          front   70.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2864  Topic10    2.011790         member    4.434585   2.5537  -5.6277
589   Topic10    2.011795           save    4.585555   2.5202  -5.6277
943   Topic10    1.362823      criterion    3.000973   2.5547  -6.0172
2794  Topic10    1.362824         freely    3.007219   2.5527  -6.0172
1620  Topic10    2.660755         search    6.635653   2.4303  -5.3481
41    Topic10    3.309706          honor    8.597811   2.3895  -5.1299
2823  Topic10    1.888673        handset    4.505060   2.4748  -5.6909
1446  Topic10    2.011788  communication    4.956527   2.4424  -5.6277
246   Topic10    2.011788        message    5.051822   2.4234  -5.6277
2114  Topic10    2.011790          spend    5.407298   2.3554  -5.6277
1964  Topic10    2.011788        enabled    5.790882   2.2869  -5.6277
133   Topic10    3.958688           work   20.293678   1.7097  -4.9508
2389  Topic10    3.238759            let   15.281777   1.7926  -5.1516
2983  Topic10    2.543355       location   